In [1]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from lib.mutation_util import get_tx_datetime, is_night, is_weekend, get_card_spending_behaviour_features, \
    get_count_risk_rolling_window, get_diff_tx_time, is_diff_previous

df_ori = pd.read_csv('../adyen-dataset.csv')


df_baseline_features = df_ori.copy()[
    ["psp_reference", "eur_amount", "card_number", "email_address", "ip_address", "ip_country","zip_code","has_fraudulent_dispute"]]
df_baseline_features["tx_datetime"] = df_ori.parallel_apply(get_tx_datetime, axis=1)  # for each row of df
df_baseline_features["tx_datetime"] = pd.to_datetime(df_baseline_features["tx_datetime"])
df_baseline_features["is_night"] = df_baseline_features.tx_datetime.parallel_apply(
    is_night)  # series.apply for each cell
df_baseline_features["is_weekend"] = df_baseline_features.tx_datetime.parallel_apply(is_weekend)
df_baseline_features = df_baseline_features.groupby('card_number').parallel_apply(
    lambda x: get_card_spending_behaviour_features(x, windows_size_in_days=[1, 7, 30]))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)
df_baseline_features = df_baseline_features.groupby('email_address', dropna=False).parallel_apply(
    lambda x: get_count_risk_rolling_window(x, feature="email_address", delay_period=7,
                                            windows_size_in_days=[1, 7, 30]))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)
df_baseline_features = df_baseline_features.groupby('ip_address', dropna=False).parallel_apply(
    lambda x: get_count_risk_rolling_window(x, feature="ip_address", delay_period=7, windows_size_in_days=[1, 7, 30]))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)
df_baseline_features = df_baseline_features.groupby('card_number').parallel_apply(lambda x: get_diff_tx_time(x))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)
df_baseline_features = df_baseline_features.groupby('card_number').parallel_apply(lambda x: is_diff_previous(x, feature="ip_country"))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)
df_baseline_features = df_baseline_features.groupby('card_number').parallel_apply(lambda x: is_diff_previous(x, feature="zip_code"))
df_baseline_features = df_baseline_features.sort_values('tx_datetime').reset_index(drop=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


ZeroDivisionError: float division by zero